In [2]:
from temp import *

In [12]:
class Generator(nn.Module):
    def __init__(self,z_dim,y_dim,h_dim):
        super(Generator,self).__init__()
        
        self.layer = nn.Sequential(
            nn.Linear(z_dim + y_dim, h_dim*4),
            Unflatten(C=h_dim, W=2, H=2),
            *block_t(h_dim,h_dim//2),
            *block_t(h_dim//2,h_dim//4),
            *block_t(h_dim//4,h_dim//8),
            *block_t(h_dim//8,1,nn.Sigmoid())
        )
        
        initialize_weights(self)
    
    def forward(self,z,y):
        return self.layer(T.cat([z,y],1))
    
class Encoder(nn.Module):
    def __init__(self,z_dim,y_dim,h_dim):
        super(Encoder,self).__init__()

        self.z_dim = z_dim
        
        self.layer = nn.Sequential(
            *block(1,h_dim//8),#32>16
            *block(h_dim//8,h_dim//4),#16>8
            *block(h_dim//4,h_dim//2),#8>4
            *block(h_dim//2,h_dim),#4>2
            Flatten(),
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(h_dim*4+y_dim, h_dim*2),
            nn.ReLU(),
            nn.Linear(h_dim*2,z_dim*2),
        )
        
        initialize_weights(self)
        
    def reparametrize(self,mu,scale):
        eps = T.randn_like(T.exp(scale))
        z = mu + eps * T.exp(scale/2)
        return z

    def forward(self,x,y):
        h = self.layer(x)
        h = self.fc_layer(T.cat([h,y],1))
        mu = h[:,:self.z_dim]
        scale = h[:,self.z_dim:]
        kl = - 0.5* T.sum(1 + scale - mu.pow(2) - scale.exp())/T.numel(mu)
        return self.reparametrize(mu,scale), kl
    
class TC_discriminator(nn.Module):
    def __init__(self,z_dim):
        super(TC_discriminator,self).__init__()
        
        self.layer = nn.Sequential(
            nn.Linear(z_dim, 512),
            nn.LeakyReLU(0.2,True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2,True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
        initialize_weights(self)
        
    def forward(self,z):
        d = self.layer(z)
        return d
        
    def g_loss(self,z):
        ones = T.ones(z.shape[0],1)
        zeros = T.zeros(z.shape[0],1)
        d_r = self.layer(z)
        loss = -F.binary_cross_entropy(d_r,ones) + F.binary_cross_entropy(d_r,zeros)
        return loss
    
    def d_loss(self,z):
        z_shf = z[:,T.randperm(z.shape[1])]
        ones = T.ones(z.shape[0],1)
        zeros = T.zeros(z.shape[0],1)
        d_r = self.layer(z)
        d_shf = self.layer(z_shf)
        loss = F.binary_cross_entropy(d_r,ones) + F.binary_cross_entropy(d_shf,zeros)
        return loss

class Discriminator(nn.Module):
    def __init__(self,h_dim):
        super(Discriminator,self).__init__()

        self.layer = nn.Sequential(
            *block(1,h_dim//8,nn.LeakyReLU(0.2)),#32>16
            *block(h_dim//8,h_dim//4,nn.LeakyReLU(0.2)),#16>8
            *block(h_dim//4,h_dim//2,nn.LeakyReLU(0.2)),#8>4
            *block(h_dim//2,h_dim,nn.LeakyReLU(0.2)),#4>2
            Flatten(),
            nn.Linear(h_dim*4, 1),
            nn.Sigmoid()
        )
        
        initialize_weights(self)

    def forward(self,x):
        return self.layer(x)

class FactorVAE_GlyphGAN(nn.Module):
    def __init__(self,E,G,D,TC,GP):
        super(FactorVAE_GlyphGAN,self).__init__()
        
        self.z_dim = E.z_dim
        self.E = E
        self.G = G
        self.D = D
        self.TC = TC
        self.GP = GP
        
    def forward(self,x,y):
        z, kl = self.E(x,y)
        x_f = self.G(z,y)
        return x_f
        
    def generate_random(self,y):
        z = T.randn(y.shape[0],self.z_dim)
        x = self.G(z,y)
        return x
    
    def calc_gp(self,x, x_p):
        alpha = T.rand(x.shape[0], 1, 1, 1).expand_as(x)

        ip = (alpha * x.data + (1 - alpha) * x_p.data)
        ip.requires_grad = True

        prob_ip = self.D(ip)

        grad = T.autograd.grad(outputs=prob_ip,
                               inputs=ip,
                               grad_outputs=T.ones(
                                   prob_ip.size()),
                               create_graph=True,
                               retain_graph=True)[0].view(x.shape[0],-1)

        grad_norm = T.sqrt(T.sum(grad ** 2, dim=1) + 1e-12)
        return self.GP * ((grad_norm - 1) ** 2).mean()
    
    def d_loss(self,x,y):
        x_p = self.generate_random(y).detach()
        z,kl = self.E(x,y)
        
        d_p = self.D(x_p).mean()
        d_r = self.D(x).mean()
        GP = self.calc_gp(x,x_p)
        d_tc = self.TC.d_loss(z)
        return {'d_p':d_p,'d_r':d_r,'GP':GP,'d_tc':d_tc}
    
    def g_loss(self,x,y):
        z, kl = self.E(x,y)
        x_f = self.G(z,y)
        x_p = self.generate_random(y)
        
        recon = (x - x_f).pow(2).mean()
        d_p = self.D(x_p).mean()
        g_tc = self.TC.g_loss(z)
        
        return {'kl':kl,'recon':recon,'d_p':d_p,'g_tc':g_tc}


In [4]:
def train_test(model,opt,loaders,writer,epoch,mode):
    loss = {i: T.tensor(0.) for i in losses}
    cnt = 0
    div = 0
    for loader in tqdm(loaders):
        for x,y in loader:
            cnt += 1
            x = x.to(device)
            y = y.to(device)
            
            ret = model.d_loss(x,y)
            d_loss = ret['d_p'] - ret['d_r'] + ret['GP'] + ret['d_tc']
            
            if mode == 'train':
                opt['D'].zero_grad()
                d_loss.backward()
                opt['D'].step()

            for i in ret.keys():
                loss[i] += ret[i]

            if cnt % n_disc == 0:
                div += 1
                ret = model.g_loss(x,y)
                g_loss = ret['kl'] + 5*ret['recon'] - ret['d_p'] + ret['g_tc']
                if mode == 'train':
                    opt['E'].zero_grad()
                    opt['G'].zero_grad()
                    g_loss.backward()
                    opt['E'].step()                    
                    opt['G'].step()
    
                for i in ret.keys():
                    if i != 'd_p':
                        loss[i] += ret[i]
    
    for i in loss.keys():
        if i == 'kl' or i == 'recon':
            loss[i] /= div
        else:
            loss[i] /= cnt
    upload(writer,loss,epoch,mode)
    
    return loss

In [5]:
root = #学習データの保存場所
models = #学習モデルの保存場所

name = 'FactorFactorVAE_GlyphGAN'
batch_size = 128
epochs = 200

seed = 1
T.manual_seed(seed)

writer = Writer(name)

y_dim = 26
z_dim = 100
h_dim = 128
n_disc = 5
GP_weight = 10
losses = ['kl','recon','d_p','d_r','GP','g_tc','d_tc']

if T.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [6]:
train_loaders, test_loaders = get_loaders(root,y_dim,batch_size)

In [13]:
E = Encoder(z_dim,y_dim,h_dim).to(device)
G = Generator(z_dim,y_dim,h_dim).to(device)
D = Discriminator(h_dim).to(device)
TC = TC_discriminator(z_dim).to(device)
model = FactorVAE_GlyphGAN(E,G,D,TC,GP_weight)
opt = {}
opt['E'] = optim.Adam(E.parameters(),lr=2e-4,betas=(0.5,0.99))
opt['G'] = optim.Adam(G.parameters(),lr=2e-4,betas=(0.5,0.99))
opt['D'] = optim.Adam(D.parameters(),lr=2e-4,betas=(0.5,0.99))

In [ ]:
label = T.zeros(64).to(T.int64).to(device)
label = F.one_hot(label, y_dim)
z_sample = T.randn(64,z_dim)
sample = G(z_sample,label)

In [15]:


for epoch in range(1+epochs, 3*epochs + 1):
    print(f'Epoch {epoch}')
    if epoch % 10 == 1:
        T.save(model.state_dict(),f'{models}/{name}.pth')
    
    train_test(model,opt,train_loaders,writer,epoch,'train')
    train_test(model,opt,test_loaders,writer,epoch,'test')
    
    sample = G(z_sample,label)
    writer.add_images('Image_from_latent', sample, epoch)
    
writer.close()




Epoch 201


100%|███████████████████████████████████████████| 26/26 [00:56<00:00,  2.18s/it]


Epoch 202


100%|███████████████████████████████████████████| 26/26 [00:56<00:00,  2.16s/it]


Epoch 203


...

Epoch 494


100%|███████████████████████████████████████████| 26/26 [01:01<00:00,  2.35s/it]


Epoch 495


 12%|█████                                       | 3/26 [00:08<01:05,  2.85s/it]


KeyboardInterrupt: 